# Doing all of the EDA and Model work

In [5]:
import numpy as np
import pandas as pd
import plotly.express as px
import torch

## Load Data

In [6]:
x = torch.rand(5, 3)
print(x)

tensor([[0.4103, 0.3823, 0.8095],
        [0.6915, 0.2055, 0.5150],
        [0.8426, 0.7800, 0.6047],
        [0.4650, 0.5369, 0.2615],
        [0.9701, 0.6494, 0.3163]])
